In [1]:
import pandas as pd
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import Ollama
import os
from PIL import Image
import io

import json
import csv
import pandas as pd 

import os
import fitz  # PyMuPDF
import faiss
import numpy as np
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import Ollama
from langchain.chains import RetrievalQA
import gradio as gr
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
def load_and_chunk_text(input_path, chunk_size, chunk_overlap):
    """Load text and metadata from a file and perform chunking."""
    with open(input_path, "r", encoding="utf-8") as f:
        extracted_data = json.load(f)

    chunks = []
    metadata = []

    for entry in extracted_data:
        text = entry["text"]
        source_metadata = {
            "source": entry["source"],
            "folder": entry["folder"],
            "file_name": entry["file_name"],
            "page": entry["page"],
            "title": entry["title"],
            "url": entry["url"]
        }

        # Chunk the text
        start = 0
        while start < len(text):
            end = start + chunk_size
            chunk = text[start:end]
            chunks.append(chunk)
            metadata.append(source_metadata)
            start += chunk_size - chunk_overlap

    return chunks, metadata


EXTRACTION_OUTPUT_PATH = r"C:\Users\LEGION\Desktop\Project\AI351\PROJECT\extracted_text.json"

# Chunking parameters
CHUNK_SIZE = 10000
CHUNK_OVERLAP = 2000
documents, metadatas = load_and_chunk_text(EXTRACTION_OUTPUT_PATH,CHUNK_SIZE,CHUNK_OVERLAP  )


In [3]:

# Initialize embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


# Create FAISS vectorstore
vectorstore = FAISS.from_texts(documents, embeddings, metadatas=metadatas)

# Initialize LLM
llm = Ollama(model="llama3.2")

# Set up Retrieval-based QA
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)

def retrieve_answer(query):
    result = qa.invoke({"query": query})
    answer = result["result"]
    source_documents = result["source_documents"]

    # Format the references
    references = []
    for i, doc in enumerate(source_documents, start=1):
        metadata = doc.metadata
        source_info = f"Source {i}: {metadata['title']} (Page {metadata['page']}, Link: {metadata['url']})"
        references.append(source_info)

    return answer, "\n".join(references)

# Generate prompts dynamically
def generate_prompt(row):
    options = []
    for choice in ['A', 'B', 'C', 'D', 'E']:
        # Check for NaN or blank values
        if pd.notna(row[choice]) and row[choice] != '':
            options.append(f"{choice}. {row[choice]}")

    # Construct the prompt with few-shot examples
    prompt = f"\nActual Question: {row['Question']}\n" + "\n".join(options)
    prompt += "\nPlease answer only in letters and put them inside a bracket '[]'. If the question contains the statement 'Check all that apply' then add comma separator if there are multiple answers ONLY IF ALLOWED."

    return prompt

# Load the Excel file
file_path = 'LTO_EXAM.csv'  # Replace with your file path
df = pd.read_csv(file_path)

# Generate prompts
df['Prompt'] = df.apply(generate_prompt, axis=1)



C:\Users\LEGION\AppData\Local\Temp\ipykernel_24160\2368169120.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


C:\Users\LEGION\AppData\Local\Temp\ipykernel_24160\2368169120.py:9: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.2")


In [13]:

def retrieve_answer(query):
    result = qa.invoke({"query": query})
    answer = result["result"]
    source_documents = result["source_documents"]

    # Format the references
    references = []
    for i, doc in enumerate(source_documents, start=1):
        metadata = doc.metadata
        source_info = f"Source {i}: {metadata['title']} (Page {metadata['page']}, Link: {metadata['url']})"
        references.append(source_info)

    return answer, "\n".join(references)


In [4]:
from tqdm import tqdm

In [14]:
qr_range = (0,60)
df["AI"] = np.nan
ai_answer = []
for i in tqdm(range(*qr_range)):
    ai_answer.append(retrieve_answer(df.loc[i,"Prompt"]))

df.loc[qr_range[0]:qr_range[1]-1, "AI"] = [answ[0] for answ in ai_answer]
df.loc[qr_range[0]:qr_range[1]-1, ["Question","Answer","AI"]]

100%|██████████| 60/60 [08:41<00:00,  8.68s/it]
C:\Users\LEGION\AppData\Local\Temp\ipykernel_24160\114560159.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '["I don't know.", '[B]', '[A]', '[A, C]', '[C]', '[3]', '[None of the above, the correct answer is not listed]', '[A]', '[B]', '[B]', "[None, I don't know]", '[ B, C ]', '[B]', '[3]', '[B]', '[C]', '[C]', '[ C ]', '[B]', '[C]', '[ C ]', '[A]', '[ C ]', '[B]', '[C]', "[ C ]\n\n(Note: The speed limit on an expressway can vary depending on the country, state, or region, but generally, a speed of 60 kph is considered moderate for an expressway. Driving at 80 kph may be allowed in some areas, but it's not necessarily safe driving everywhere. Changing lanes without signaling is a safety hazard regardless of the speed.)", 'A]', '[A]', '[A] [B]', "I don't know.", "I don't see any information about backing up or checking answers in the provided text. The 

,Question,Answer,AI
0,What should you do in case your vehicle breaks...,"A, C, D, E",I don't know.
1,What will happen when your front tire blows out?,C,[B]
2,What should you do when an ambulance comes up ...,D,[A]
3,While driving the hood of your car lifts up bl...,"A,C","[A, C]"
4,"In case of an accident, the first duty of the ...",A,[C]
5,"When a vehicle starts to skid, what should the...",B,[3]
6,"In case of injuries caused by an accident, the...",A,"[None of the above, the correct answer is not ..."
7,What will happen when your rear tire blows out?,B,[A]
8,"When a vehicle is stalled or disabled, the dri...",C,[B]
9,If you are the first to arrive at the scene of...,B,[B]


In [16]:
df.to_csv('RESULT_V1.csv',index=False)